<a href="https://colab.research.google.com/github/tribendu/LLM-fine-tunig/blob/main/Working_llama_3%2C2_3B_US_overfitted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install necessary dependencies

In [1]:
!pip install -q pydrive2 transformers datasets unsloth bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Import necessary libraries and functions

In [19]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth, drive
from oauth2client.client import GoogleCredentials
import json
from transformers import LlamaForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch
from datasets import load_dataset, Dataset,load_from_disk
from unsloth.chat_templates import train_on_responses_only, get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer
from sklearn.model_selection import train_test_split
from unsloth import FastLanguageModel
from transformers import LlamaTokenizer, LlamaForCausalLM, DataCollatorForLanguageModeling

In [3]:


# Authenticate and mount Google Drive
def authenticate_and_mount_drive():
    auth.authenticate_user()
    drive.mount('/content/drive')
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    return GoogleDrive(gauth)

def convert_to_huggingface_format(input_file_path, output_file_path):
    # Read the original JSON file
    try:
        with open(input_file_path, 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: The file {input_file_path} was not found.")
        return
    except json.JSONDecodeError:
        print(f"Error: The file {input_file_path} could not be decoded as JSON.")
        return

    # Convert to HuggingFace's generic format
    huggingface_data = []
    for entry in data:
        huggingface_data.append({"role": "user", "content": f"{entry['instruction']}\n\n{entry['input']}"})
        huggingface_data.append({"role": "assistant", "content": entry['output']})

    # Print the first 3 rows of the formatted data
    print("First 3 rows of the formatted data:")
    print(huggingface_data[:3])

    # Write to the new JSON file
    try:
        with open(output_file_path, 'w') as f:
            for item in huggingface_data:
                f.write(json.dumps(item) + '\n')
        print(f"Formatted data saved to {output_file_path}")
    except IOError as e:
        print(f"Error: Could not write to file {output_file_path}. {e}")

def upload_file_to_drive(drive, file_path, new_file_name):
    # Check if a file with the same name already exists in Google Drive
    existing_files = drive.ListFile({'q': f"title = '{new_file_name}' and trashed = false"}).GetList()
    if existing_files:
        print(f"Warning: A file with the name '{new_file_name}' already exists. Consider renaming or deleting the existing file to prevent accidental overwrites.")
        return

    # Upload the new file to Google Drive
    new_file = drive.CreateFile({'title': new_file_name})
    new_file.SetContentFile(file_path)
    new_file.Upload()
    print(f"Uploaded file to Google Drive with name: {new_file_name}")

**Mounting google drive and formatting to huggingface generic format**

In [4]:
drive_client = authenticate_and_mount_drive()

    # Set up file details
original_file_path = "/content/drive/My Drive/Fine-tune/training_data_US.json"
formatted_file_path = "/content/drive/My Drive/Fine-tune/formatted_hf_training_data.json"
new_file_name = "hf_training_data.json"
# Convert the dataset to the HuggingFace format
convert_to_huggingface_format(original_file_path, formatted_file_path)
# Upload the formatted file back to Google Drive
upload_file_to_drive(drive_client, formatted_file_path, new_file_name)


Mounted at /content/drive
First 3 rows of the formatted data:
[{'role': 'user', 'content': 'Create a Jira user story for a financial advisor platform.\n\nUser Role: Financial Advisor, Task: Generate financial reports for clients'}, {'role': 'assistant', 'content': 'As a Financial Advisor, I want to generate financial reports for my clients so that I can provide them with a clear understanding of their financial progress.\n\nAcceptance Criteria:\nPositive Scenarios:\n- Given the financial reporting tool, when I select a client’s data, then a report should be generated summarizing their financial performance.\n- Given customizable report templates, when I create reports, then I should be able to choose the format and metrics most relevant to the client’s goals.\n- Given periodic reporting, when I set a schedule, then reports should be generated and sent to clients automatically.\n- Given historical data, when generating a report, then it should include year-over-year performance comparis

#Set up model parameters for FastLanguageModel

In [10]:
max_seq_length = 1024 # Reduced sequence length for T4 GPU optimization.
dtype = torch.float16 # Set dtype to float16 for optimal performance on Tesla T4.
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

Install libraries

In [15]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.

In [11]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Apply PEFT model settings

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# Set up chat template for tokenizer

In [13]:

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Create train and eval datasets from formatted data

In [14]:

def create_train_eval_datasets(formatted_file_path, train_output_path, eval_output_path, test_size=0.2):
    try:
        with open(formatted_file_path, 'r') as f:
            lines = f.readlines()
            data = [json.loads(line) for line in lines]
    except FileNotFoundError:
        print(f"Error: The file {formatted_file_path} was not found.")
        return
    except json.JSONDecodeError:
        print(f"Error: The file {formatted_file_path} could not be decoded as JSON.")
        return

    dataset = Dataset.from_dict({"text": [item["content"] for item in data]})
    split_dataset = dataset.train_test_split(test_size=test_size)
    train_dataset = split_dataset['train']
    eval_dataset = split_dataset['test']

    # Save train and eval datasets to disk
    train_dataset.save_to_disk(train_output_path)
    eval_dataset.save_to_disk(eval_output_path)
    print(f"Train dataset saved to {train_output_path}")
    print(f"Eval dataset saved to {eval_output_path}")

# Set up paths for train and eval datasets
train_output_path = "/content/drive/My Drive/Fine-tune/train_dataset"
eval_output_path = "/content/drive/My Drive/Fine-tune/eval_dataset"

# Create train and eval datasets
create_train_eval_datasets(formatted_file_path, train_output_path, eval_output_path)

Saving the dataset (0/1 shards):   0%|          | 0/62 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/16 [00:00<?, ? examples/s]

Train dataset saved to /content/drive/My Drive/Fine-tune/train_dataset
Eval dataset saved to /content/drive/My Drive/Fine-tune/eval_dataset


#Fine-tune LLAMA model using SFTTrainer from TRL

In [20]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=load_from_disk(train_output_path),
    eval_dataset=load_from_disk(eval_output_path),
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding='longest'), # Dynamic padding for efficiency
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=1,  # Reduced batch size for T4 GPU
        gradient_accumulation_steps=8,  # Higher gradient accumulation to compensate for lower batch size
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=True,  # Use FP16 for better performance on T4
        logging_steps=1,
        optim="adamw_8bit", # 8-bit optimizer for reduced memory usage
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/62 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/16 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [22]:
def preprocess_function(examples):
    inputs = tokenizer(examples["text"], max_length=max_seq_length, truncation=True, padding='max_length')
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

# Apply preprocessing to train and eval datasets
train_dataset = load_from_disk(train_output_path).map(preprocess_function, batched=True)
eval_dataset = load_from_disk(eval_output_path).map(preprocess_function, batched=True)

# Update trainer with preprocessed datasets
trainer.train_dataset = train_dataset
trainer.eval_dataset = eval_dataset

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [23]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 62 | Num Epochs = 9
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,13.375100
2,13.362300
3,13.190900
4,11.399800
5,8.731000
6,8.903400
7,8.189400
8,8.170300
9,7.853200
10,6.816400


TrainOutput(global_step=60, training_loss=6.172155030568441, metrics={'train_runtime': 679.8958, 'train_samples_per_second': 0.706, 'train_steps_per_second': 0.088, 'total_flos': 9546489239961600.0, 'train_loss': 6.172155030568441, 'epoch': 7.741935483870968})

# Save the fine-tuned model

In [24]:

model_output_path = "/content/drive/My Drive/llama_finetuned_model"
model.save_pretrained(model_output_path)
tokenizer.save_pretrained(model_output_path)
print(f"Fine-tuned model saved to {model_output_path}")

Fine-tuned model saved to /content/drive/My Drive/llama_finetuned_model


# Now, let's test this model

Load the model

In [26]:
# Load the fine-tuned model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the fine-tuned model
model_output_path = "/content/drive/My Drive/llama_finetuned_model"
model = AutoModelForCausalLM.from_pretrained(model_output_path)
tokenizer = AutoTokenizer.from_pretrained(model_output_path)

# Move model to the correct device (e.g., GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


`low_cpu_mem_usage` was None, now set to True since model is quantized.


#Funcion to generate responses

In [29]:
import torch

def generate_response(prompt, model, tokenizer, max_length=150):
    # Tokenize input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate response using the model
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id  # Ensure the model can pad correctly
        )

    # Decode the output tokens and return the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example prompt to test your fine-tuned model
prompt = "Create a Jira user story for a financial advisor platform.\n\nUser Role: Financial Advisor, Task: Generate financial reports for clients."
response = generate_response(prompt, model, tokenizer)
print("Model Response:", response)


AttributeError: 'LlamaForCausalLM' object has no attribute 'max_seq_length'

In [32]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from unsloth import FastLanguageModel

# Load the fine-tuned model and tokenizer
model_output_path = "/content/drive/My Drive/llama_finetuned_model"

# Load the model using Unsloth's FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(model_output_path)

# Move model to the correct device (e.g., GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Call for_inference before performing inference
FastLanguageModel.for_inference(model)

def generate_response(prompt, model, tokenizer, max_length=150):
    # Tokenize input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate response using the model
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id  # Ensure the model can pad correctly
        )

    # Decode the output tokens and return the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response



==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model Response: Create a Jira user story for a financial advisor platform.

User Role: Financial Advisor, Task: Generate financial reports for clients. Acceptance Criteria:
Positive Scenarios:
- Given the financial advisor platform, when generating reports, then the system should allow me to select report types (income, expenses, investments, etc.) and customize the output.
- Given the ability to generate reports, when I select a client, then the system should automatically populate the report with their specific financial data.
- Given multipl

#Test 1: Example prompt to test your fine-tuned model
prompt = "Create a Jira user story for a financial advisor platform.\n\nUser Role: Financial Advisor, Task: Generate financial reports for clients."


In [34]:
prompt = "Create a Jira user story for a financial advisor platform.\n\nUser Role: Financial Advisor, Task: Generate financial reports for clients."
response = generate_response(prompt, model, tokenizer)
print("Model Response:\n", response)

Model Response:
 Create a Jira user story for a financial advisor platform.

User Role: Financial Advisor, Task: Generate financial reports for clients. Acceptance Criteria:
Positive Scenarios:
- Given a dashboard, when generating reports, then the system should automatically create and display client-specific financial reports.
- Given various report formats (e.g., PDF, Excel), when generating reports, then the system should offer these options and let the user select the preferred format.
- Given a client’s financial history, when generating reports, then the system should include all relevant data and trends.
- Given multiple clients, when generating reports, then the system should allow filtering and prioritizing the clients based on user preferences.
- Given real-time data, when generating reports, then the


In [36]:
prompt = "Why is the sky blue?\n answer in one sentence."
response = generate_response(prompt, model, tokenizer)
print("Model Response:\n", response)

Model Response:
 Why is the sky blue?
 answer in one sentence. The sky appears blue because when sunlight enters Earth's atmosphere, shorter blue wavelengths scatter more than other wavelengths, giving the sky its blue hue.
This response meets the one-sentence requirement, but it's a bit too brief. Consider expanding it to provide more context and clarity. Here's a revised version:
The sky appears blue because when sunlight enters Earth's atmosphere, shorter blue wavelengths scatter more than other wavelengths due to a phenomenon called Rayleigh scattering, which occurs when sunlight interacts with the tiny molecules of gases such as nitrogen and oxygen in the atmosphere. As a result, the scattered blue light is dispersed throughout the atmosphere, giving the sky its characteristic blue color.
